In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import glob, pylab, pandas as pd
import pydicom, numpy as np

# Any results you write to the current directory are saved as output.

In [ ]:
#reading the data and splitting it into positive and negative cases!!!
#getting the unique positive cases
df = pd.read_csv('../input/stage_2_train_labels.csv')
print(df.shape)
negative=df[df.Target==0]
positive=df[df.Target==1]
pcases=positive.patientId.unique()
print(negative.shape,positive.shape,pcases.shape)

In [ ]:
#reading the ground truth and structuring it 
xywh_cases=[]
ratios=[]
sizes=[]
for p in pcases:
    print(p)
    xywh_case=[]
    for i in range(len(positive[positive.patientId==p])):
        xywh=[]
        xywh.append(positive[positive.patientId==p].x.iloc[i])
        xywh.append(positive[positive.patientId==p].y.iloc[i])
        xywh.append(positive[positive.patientId==p].width.iloc[i])
        xywh.append(positive[positive.patientId==p].height.iloc[i])
        xywh_case.append(xywh)
        ratios.append(positive[positive.patientId==p].height.iloc[i]/positive[positive.patientId==p].width.iloc[i])
        sizes.append(positive[positive.patientId==p].height.iloc[i]*positive[positive.patientId==p].width.iloc[i])
    xywh_cases.append(xywh_case) 

In [ ]:
ratios=np.array(ratios)
sizes=np.array(sizes)
sorted_sizes=sorted(sizes)
print(sorted_sizes)

In [ ]:
img_data=[]
for p in pcases:
    dcm_file = '../input/stage_2_train_images/%s.dcm' % p
    dcm_data = pydicom.read_file(dcm_file)
    img_data.append(dcm_data.pixel_array)
img_data=np.expand_dims(img_data,axis=-1)

In [ ]:
def intersection_area (gt,anchors):
    x1_min,y1_min,x1_max,y1_max=gt
    x2_min,y2_min,x2_max,y2_max=anchors
    
    x_min_max=max(x1_min,x2_min)
    y_min_max=max(y1_min,y2_min)
    x_max_min=min(x1_max,x2_max)
    y_max_min=min(y1_max,y2_max)
    
    return((x_max_min-x_min_max)*(y_max_min-y_min_max))

def union_area(gt,anchors):
    x1_min,y1_min,x1_max,y1_max=gt
    x2_min,y2_min,x2_max,y2_max=anchors
    
    x_min=min(x1_min,x2_min)
    y_min=min(y1_min,y2_min)
    x_max=max(x1_max,x2_max)
    y_max=max(y1_max,y2_max)
    
    return((x_max-x_min)*(y_max-y_min))

def iou(gt,anchors):
    unionArea=union_area(gt,anchors)
    intersectionArea=intersection_area(gt,anchors)
    print(unionArea,intersectionArea)
    iou=intersectionArea/(unionArea+0.000000000000000000000000000000000001)
    return(iou)
    
i=iou([0, 0, 100, 100], [0, 0, 100, 100])
print(i)

In [ ]:
iou

In [ ]:
import math
import numpy as np
boxes=[152.0, 264.0, 379.0, 213.0]
anchor_ratios=((1,1),(1,1/math.sqrt(2)),(1/math.sqrt(2),1),(1,2),(2,1))
anchor_ratios=np.array(anchor_ratios)
noc=2 #no. of classes including background

input_size=(1024,1024)
fm_1_classifier=np.zeros((32,32,noc))
fm_1_position =np.zeros((32,32,len(anchor_ratios)))
fm_1_regression=np.zeros((32,32,4))


x_position=np.arange(32)
y_position=np.arange(32)

x_position_upscaled=(x_position+0.5)*(1024/32)
y_position_upscaled=(y_position+0.5)*(1024/32)

x_left=np.multiply.outer([x_position_upscaled-(32/2)],(anchor_ratios[:,0]))
y_up=np.multiply.outer([y_position_upscaled-(32/2)],anchor_ratios[:,1])
x_right=np.multiply.outer([x_position_upscaled+(32/2)],anchor_ratios[:,0])
y_down=np.multiply.outer([y_position_upscaled+(32/2)],anchor_ratios[:,1])

#x_left=x_position_upscaled-(32/2)
#y_up =y_position_upscaled-(32/2)
#x_right=x_position_upscaled+(32/2)
#y_down=y_position_upscaled+(32/2)



In [ ]:
y_up.shape

In [ ]:
all_boxes=[]
for r in range(32):
    for c in range(32):
        for a in range(len(anchor_ratios)):
            all_box=[]
            all_box.append(x_left[0][r][a])
            all_box.append(y_up[0][c][a])
            all_boxes.append(all_box)

In [ ]:
all_boxes

In [ ]:
x_left_repeated=np.repeat(x_left,32)
x_left_repeated=x_left_repeated.reshape((32,32,5))
x_left_repeated[1]

In [ ]:
np.random.seed(23)
a=np.random.randint(9,size=(3,2))
b=np.random.randint(10,19,size=(4,2))

aa=np.repeat(a,4,axis=1)
#aa=aa.reshape((3,4,2))
bb=np.tile(b,(3,1))
print(a,b)
print(aa,bb.shape)
print(bb)

In [ ]:
a=np.random.randint(5,size=(3,2))
b=np.random.randint(10,13,size=(4,2))


aa=np.repeat(a,(6))
aa=aa.reshape((5,6))

bb=np.tile(b,(5,1))

print(aa.shape,bb.shape)
aa=np.expand_dims(aa,axis=-1)
bb=np.expand_dims(bb,axis=-1)
c=np.concatenate((aa,bb),axis=-1)
#print(bb,aa)
print(c)

In [ ]:
y_repeated=np.repeat(y_position,32)
y_repeated=y_repeated.reshape((32,32))
print(y_repeated[1])
z=np.concatenate((np.expand_dims(x_position,axis=1),y_repeated),axis=1)
print(z.shape)
z[0]

In [ ]:
#creating the model
import keras
from keras.models import Model
from keras.layers import Input,Conv2D, Reshape,Concatenate
from keras.regularizers import l2
l2_reg=0.0005

x= Input(shape=(1024,1024,1))
conv1 = Conv2D(3, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv1')(x)
conv2 = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv2')(conv1)
conv3 = Conv2D(16, (3, 3), activation='relu', padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv3')(conv2)
conv4 = Conv2D(16, (3, 3),strides=2, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv4')(conv3)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv5')(conv4)
conv6 = Conv2D(32, (3, 3),strides=2, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv6')(conv5)
conv7 = Conv2D(64, (3, 3), activation='relu', padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv7')(conv6)
conv8 = Conv2D(64, (3, 3),strides=2, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv8')(conv7)
conv9 = Conv2D(128, (3, 3), activation='relu', padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv9')(conv8)
conv10 = Conv2D(256, (3, 3),strides=2, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv10')(conv9)
conv11 = Conv2D(512, (3, 3),strides=2, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11')(conv10)

conv12 = Conv2D(512, (3, 3),strides=2, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12')(conv11)

conv13 = Conv2D(512, (3, 3),strides=2, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13')(conv12)

conv11a = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11a')(conv11)
conv11b = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11b')(conv11a)
conv11c = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11c')(conv11b)
conv11d = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11d')(conv11c)
conv11e = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11e')(conv11d)
conv11f = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11f')(conv11e)
conv11g = Conv2D(2, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11g')(conv11f)

conv12a = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12a')(conv12)
conv12b = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12b')(conv12a)
conv12c = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12c')(conv12b)
conv12d = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12d')(conv12c)
conv12e = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12e')(conv12d)
conv12f = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12f')(conv12e)
conv12g = Conv2D(2, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12g')(conv12f)

conv13a = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13a')(conv13)
conv13b = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13b')(conv13a)
conv13c = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13c')(conv13b)
conv13d = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13d')(conv13c)
conv13e = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13e')(conv13d)
conv13f = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13f')(conv13e)
conv13g = Conv2D(2, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13g')(conv13f)

flattened_conv11g=Reshape(((32*32), 2), name='flattened_conv_11')(conv11g)
flattened_conv12g=Reshape(((16*16), 2), name='flattened_conv_12')(conv12g)
flattened_conv13g=Reshape(((8*8), 2), name='flattened_conv_13')(conv13g)

classifier_concatenated=Concatenate(axis=1,name="classifier_concatenated")([flattened_conv11g,flattened_conv12g,flattened_conv13g])


conv11A = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11A3G')(conv11)
conv11B = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11B')(conv11A)
conv11C = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11C')(conv11B)
conv11D = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11D')(conv11C)
conv11E = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11E')(conv11D)
conv11F = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11F')(conv11E)
conv11G = Conv2D(4, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11G')(conv11F)

conv12A = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12A')(conv12)
conv12B = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12B')(conv12A)
conv12C = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12C')(conv12B)
conv12D = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12D')(conv12C)
conv12E = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12E')(conv12D)
conv12F = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12F')(conv12E)
conv12G = Conv2D(4, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12G')(conv12F)

conv13A = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13A')(conv13)
conv13B = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13B')(conv13A)
conv13C = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13C')(conv13B)
conv13D = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13D')(conv13C)
conv13E = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13E')(conv13D)
conv13F = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13F')(conv13E)
conv13G = Conv2D(4, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13G')(conv13F)

flattened_conv11G=Reshape(((32*32), 4), name='flattened_conv_11_reg')(conv11G)
flattened_conv12G=Reshape(((16*16), 4), name='flattened_conv_12_reg')(conv12G)
flattened_conv13G=Reshape(((8*8), 4), name='flattened_conv_13_reg')(conv13G)

regressor_concatenated=Concatenate(axis=1,name="regressor_concatenated")([flattened_conv11G,flattened_conv12G,flattened_conv13G])


model = Model(inputs=x, outputs=[classifier_concatenated,regressor_concatenated])


In [ ]:
print(classifier_concatenated.shape)


In [ ]:
for p in pcases:
    print(p)
    print(len(positive[positive.patientId==p]))

In [ ]:
print(img_data[0].shape)
img_datanp=np.array(img_data)
print(img_datanp.shape)

In [ ]:
df1=[]
for d in range(df.shape[0]):
    
    df1.append(df.iloc[d][0])
df1=np.array(df1)
print(df1.shape)

In [ ]:
udf1,counts=np.unique(df1,return_counts=True)
print(udf1.shape,counts.shape)
print(np.unique(counts))

In [ ]:
Rows=[]
columns=[]
training_data=[]
for i in range((df.shape[0])):
    patientId = df['patientId'][i]
    dcm_file = '../input/stage_2_train_images/%s.dcm' % patientId
    dcm_data = pydicom.read_file(dcm_file)
    training_data.append(dcm_data.pixel_array)
    Rows.append(dcm_data.Rows)
    columns.append(dcm_data.Columns)
rowsu=np.unique(Rows)
columnsu=np.unique(columns)